PyGotham Talk, 10/6/17

In [31]:
## 1
from lxml import html
from utils import *
import requests
import calendar
import json
import string
import re

## 2
from __future__ import print_function
from PIL import Image
#brew install tesseract
import pytesseract
import sys

## 3

## 4
import spacy
nlp = spacy.load('en')

import urllib
import functools
import itertools
import numpy as np
import nltk
#nltk.download()
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stemmer = PorterStemmer()

from PyDictionary import PyDictionary
dictionary=PyDictionary()

## 5
import pandas as pd

In [15]:
def sortVocab(maxlen):
    sortedvocab = {}
    keys = []
    for i in [w for w in nlp.vocab if w.has_vector and w.orth_ == re.sub('[^A-Za-z]','',w.orth_) and w.orth_.islower() and len(w.orth_) <= maxlen]:
        k = len(i.orth_)
        if k not in keys:
            sortedvocab[k] = []
            keys.append(k)
        sortedvocab[k].append(i)
    return sortedvocab
cosine = lambda v1, v2: np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
vocab = sortVocab(15)

# Solving a Crossword Puzzle the Hard Way
## Alec Barrett
### PyGotham
### October 7, 2017

### Outline
1. NYT Crossword
2. The Task
3. NLP
4. My Solver
5. Conclusion

## I. New York Times Crossword

### Puzzle features
* American-style
* Get more difficult as the week goes on
* 15 x 15 every day except Sunday (21 x 21)
* ~75-80 clue/answer pairs

### Relevant skills
* Large vocabulary
* Encyclopedic knowledge
* Lateral thinking/word association
* Filter by length and known letters

## II. The Task

### Computer strengths...
* Searching a corpus for a word
* Filtering a list based on a set of conditions

### ...and weaknesses
* Interpreting puns
* Considering pairs of answers together

### Strategy
* Generate a list of candidates
* Identify the correct answer based on intersecting letters

### Sources of candidates
* Google Knowledge Graph search API
* Wikipedia search API
* Dictionary (PyDictionary)
* WordNet (NLTK)
* Word Vectors (spaCy)

### Sources of candidates
#### Encyclopedias
* Google Knowledge Graph search API
* Wikipedia search API

#### Dictionaries
* Dictionary (PyDictionary)
* WordNet (NLTK)
* Word Vectors (spaCy)

### When to use each
|Condition |Example|Tools
|----------             |--------|-----
|References another cell|`Slippery 1-Across`|*Skip*
|Single word|`Stops`|Dictionary, WordNet, Vectors
|Space in the clue|`___ of Sandwich`|KG, Wiki, Vectors
|Else|`Gas company famous for its toy trucks`|KG, Wiki, Vectors
|+if we can extract one word|`Distort, as data`|+Dictionary, WordNet

## III. Natural Language Processing

### Two NLP Libraries
1. **Wordnet (NLTK)**
2. Word Vectors (spaCy)

### NLTK: Relationships between words
* Hypernyms (less specific)
* Root hypernyms (much less specific)
* Hyponyms (more specific)
* Member holonyms (more/equally specific)

In [43]:
def noSpace(word):
    return re.sub('_','',word)

def searchWordnetSpaceDemo(synset, length, candidates=set(), iteration=2):
    print('--> ',synset,"(iteration",iteration,")")
    if synset.lemma_names():
        candidates.update({ 
            noSpace(lemmaname) for lemmaname in synset.lemma_names() if noSpace(lemmaname) == length 
        })
    for attr in ['root_hypernyms','member_holonyms','hyponyms','hypernyms']:
        if getattr(synset,attr)():
            for nym in getattr(synset,attr)():
                results = { noSpace(lemma.name()) for lemma in nym.lemmas() if len(noSpace(lemma.name())) == length }
                if len(results):
                    print(attr,results)
                if iteration < 3:
                    searchWordnetSpaceDemo(nym,length,candidates,iteration+1)

In [44]:
synonyms = wn.synsets('somersault')
print(len(synonyms), synonyms)
for syn in synonyms:
    searchWordnetSpaceDemo(syn,4)

2 [Synset('somersault.n.01'), Synset('somersault.v.01')]
-->  Synset('somersault.n.01') (iteration 2 )
-->  Synset('entity.n.01') (iteration 3 )
-->  Synset('flip-flop.n.04') (iteration 3 )
hypernyms {'flip'}
-->  Synset('tumble.n.01') (iteration 3 )
hyponyms {'flip'}
-->  Synset('somersault.v.01') (iteration 2 )
root_hypernyms {'move'}
-->  Synset('move.v.03') (iteration 3 )
root_hypernyms {'move'}
hyponyms {'stir'}
hyponyms {'take'}
hyponyms {'beat'}
hyponyms {'flap', 'beat'}
hyponyms {'bolt'}
hyponyms {'buck', 'jerk'}
hyponyms {'cant', 'tilt'}
hyponyms {'tilt'}
hyponyms {'chop'}
hyponyms {'roil', 'boil', 'moil'}
hyponyms {'duck'}
hyponyms {'exit'}
hyponyms {'flex', 'bend'}
hyponyms {'funk'}
hyponyms {'flip'}
hyponyms {'flux', 'flow'}
hyponyms {'grab'}
hyponyms {'jerk'}
hyponyms {'jolt'}
hyponyms {'jump', 'leap'}
hyponyms {'jump', 'leap'}
hyponyms {'lean', 'list'}
hyponyms {'hurl'}
hyponyms {'mill'}
hyponyms {'mope'}
hyponyms {'give'}
hyponyms {'beat'}
hyponyms {'roll', 'wave', 'flap

### NLTK: Stopwords
* Remove common words

In [32]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

### NLTK: Parts of Speech
* Extract most relevant information
* Determine case/tense of answer

In [6]:
print(nltk.pos_tag(nltk.word_tokenize("Butcher's offerings")))

[('Butcher', 'NNP'), ("'s", 'POS'), ('offerings', 'NNS')]


In [7]:
print(nltk.pos_tag(nltk.word_tokenize("butcher's offerings")))

[('butcher', 'NN'), ("'s", 'POS'), ('offerings', 'NNS')]


In [8]:
clue = "Jump in an ice rink"
[nlp.vocab[x.lemma_].lower_ for x in nlp(clue.lower()) if x.tag_ in ["NNS","NNPS","NN","NNP"]]

['jump', 'ice', 'rink']

### Two NLP Libraries
1. Wordnet (NLTK)
2. **Word Vectors (spaCy)**

### spaCy: Word Vectors
* Million-word dictionary
* GloVe
* 300-element numpy array

### spacy: Word Vectors
* `King - Man + Woman = Queen`

In [9]:
nlp('king').vector

array([  3.15420002e-01,  -3.50679994e-01,   4.29230005e-01,
        -5.38250029e-01,  -1.84799999e-01,  -3.10820013e-01,
         2.91960001e-01,  -7.10300028e-01,  -2.38670006e-01,
         1.84710002e+00,  -3.64459991e-01,  -5.12820005e-01,
         1.22100003e-01,   3.89090002e-01,  -7.32040033e-02,
         3.54619995e-02,   3.32890004e-01,   6.64659977e-01,
         2.71749999e-02,   4.20210004e-01,  -1.45199999e-01,
         3.79909992e-01,  -6.05199993e-01,   1.06950000e-01,
        -6.47159994e-01,  -1.07389996e-02,  -3.97540003e-01,
         3.88570011e-01,  -2.01340005e-01,   6.98130012e-01,
        -3.24110001e-01,   7.30849981e-01,  -1.09300002e-01,
        -2.35110000e-01,   1.84819996e-01,  -1.15950003e-01,
        -7.10030019e-01,  -2.29739994e-01,  -4.19790000e-01,
         8.10039975e-03,  -1.05039999e-01,  -4.48020011e-01,
        -7.39279985e-02,  -4.23799992e-01,   2.84819990e-01,
        -7.45169967e-02,   9.81609970e-02,   6.46019995e-01,
        -2.58320004e-01,

In [10]:
queen1 = nlp('king').vector - nlp('man').vector + nlp('woman').vector
queen2 = nlp('queen').vector
queen1 - queen2

array([ 0.10458702, -0.05152999, -0.01085299,  0.40603995,  0.111525  ,
        0.03181005, -0.18277001,  0.10793996,  0.22586   ,  0.42549992,
       -0.62051803,  0.09305897, -0.0758817 , -0.29067168, -0.29784101,
       -0.43369001, -0.44859397,  0.21168   , -0.17273501,  0.24211   ,
        0.20211001, -0.15502006, -0.04844499, -0.202636  , -0.21129996,
        0.45776799,  0.03138995,  0.13294101, -0.53480601, -0.07134694,
       -0.157518  , -0.05403006, -0.14246997, -0.77390599,  0.15866998,
       -0.12601201, -0.19204   , -0.40347007,  0.05978   ,  0.52036041,
        0.37191999, -0.252379  , -0.097138  , -0.40504098,  0.25123   ,
       -0.03785798, -0.11933102, -0.00672996,  0.40257999,  0.02721703,
       -0.29956898,  0.34834102, -0.15371901, -0.14056298,  0.17291501,
        0.73967993, -0.0257776 , -0.28438202, -0.33745399,  0.12431702,
        0.063307  , -0.39151499, -0.24294749,  0.3378177 ,  0.37893206,
        0.14127994,  0.70388097,  0.021424  ,  0.142003  ,  0.20

### spacy: Word Vectors
* Find n closest vectors in our vocabulary
* Vocabulary is already nested by word length
* `cosine = lambda v1, v2: np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))`

In [45]:
def getSpacyFormulationsDemo(clue):
    formulations = []
    formulations.append([nlp.vocab[x] for x in clue.split() if x not in stopwords.words('english')])
    formulations.append([nlp.vocab[x.lower_] for x in nlp(clue) if x.pos_ == "NOUN" or x.pos_ == "PROPN"])
    formulations.append([nlp.vocab[x.lower_] for x in nlp(clue) if x.pos_ in ["NOUN","PROPN","ADJ","ADV"] ])
    formulations.append([nlp.vocab[x.lower_] for x in nlp(clue) if x.pos_ is not "PART"])
    formulations.append([nlp.vocab[x.lemma_] for x in nlp(clue.lower()) if x.tag_ in ["NNS","NNPS","NN","NNP"]])
    if clue.find(',') != -1:
        formulations += getSpacyFormulationsDemo(clue[:clue.find(',')])
    if clue.find(' (') != -1:
        formulations += getSpacyFormulationsDemo(clue[:clue.find(' (')])
    return formulations

def getSpacyCandidatesDemo(clue,length,vocab,ret_count):
    vecs = [x.vector for x in clue]
    vecsum = functools.reduce(lambda x,y: np.add(x,y),vecs)
    vocab = [w for w in vocab if w not in clue and re.search("\'", w.lower_)==None]
    vocab.sort(key=lambda w: cosine(w.vector, vecsum))
    print({w.orth_.lower() for w in vocab[-1*ret_count:]})

In [46]:
for formulation in getSpacyFormulationsDemo("Plan that's hatched"):
    print('--> ',[x.lower_ for x in formulation])
    getSpacyCandidatesDemo(formulation,6,vocab[6],10)

-->  ['plan', "that's", 'hatched']
{'agenda', 'decide', 'scheme', 'future', 'should', 'intend', 'agreed', 'policy', 'effort', 'budget'}
-->  ['plan']
{'agenda', 'decide', 'scheme', 'future', 'should', 'intend', 'agreed', 'policy', 'effort', 'budget'}
-->  ['plan', 'that']
{'decide', 'though', 'simply', 'rather', 'enough', 'future', 'should', 'really', 'likely', 'reason'}
-->  ['plan', 'that', "'s", 'hatched']
{'wanted', 'saying', 'though', 'simply', 'rather', 'enough', 'future', 'should', 'really', 'reason'}
-->  ['plan']
{'agenda', 'decide', 'scheme', 'future', 'should', 'intend', 'agreed', 'policy', 'effort', 'budget'}


In [33]:
for formulation in getSpacyFormulationsDemo("Counterpart to 'if', in computer science"):
    print('--> ',[x.lower_ for x in formulation])
    getSpacyCandidatesDemo(formulation,4,vocab[4],8)

print('\nWhen we just search for "if":')
getSpacyCandidatesDemo(nlp('if'),4,vocab[4],8)

-->  ['counterpart', "'if',", 'computer', 'science']
{'data', 'tech', 'mind', 'kind', 'arts', 'work', 'math', 'idea'}
-->  ['counterpart', 'computer', 'science']
{'data', 'tech', 'mind', 'kind', 'arts', 'work', 'math', 'idea'}
-->  ['counterpart', 'computer', 'science']
{'data', 'tech', 'mind', 'kind', 'arts', 'work', 'math', 'idea'}
-->  ['counterpart', 'to', "'", 'if', "'", ',', 'in', 'computer', 'science']
{'they', 'else', 'want', 'even', 'when', 'kind', 'what', 'that'}
-->  ['counterpart', 'computer', 'science']
{'data', 'tech', 'mind', 'kind', 'arts', 'work', 'math', 'idea'}
-->  ['counterpart', "'if'"]
{'derp', 'zerg', 'gifs', 'prob', 'ikke', 'mech', 'mais', 'mobo'}
-->  ['counterpart']
{'derp', 'zerg', 'gifs', 'prob', 'ikke', 'mech', 'mais', 'mobo'}
-->  ['counterpart']
{'derp', 'zerg', 'gifs', 'prob', 'ikke', 'mech', 'mais', 'mobo'}
-->  ['counterpart', 'to', "'", 'if', "'"]
{'they', 'else', 'will', 'even', 'want', 'when', 'make', 'that'}
-->  ['counterpart']
{'derp', 'zerg', '

/Users/alec/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in float_scalars
  # This is added back by InteractiveShellApp.init_path()


In [34]:
getSpacyCandidatesDemo(nlp('dairy animal'),3,vocab[3],10)

{'cat', 'pet', 'dog', 'eat', 'pig', 'cow', 'egg', 'soy', 'fed', 'zoo'}


In [35]:
getSpacyCandidatesDemo(nlp('dairy'),3,vocab[3],10)

{'raw', 'oil', 'eat', 'pig', 'cow', 'egg', 'hay', 'soy', 'fat', 'fed'}


In [36]:
getSpacyCandidatesDemo(nlp('animal'),3,vocab[3],10)

{'cat', 'fox', 'fur', 'toy', 'pet', 'dog', 'pig', 'cow', 'rat', 'zoo'}


## IV. My Solver

### Utility Functions
* Web scraping via BeautifulSoup
* OCR via pytesseract

### OCR via pytesseract
![Monday's Puzzle](images/1002-17.png)

In [47]:
def parseAndPrintImageDemo(n_squares, img_path):
    im = Image.open(img_path)
    im = Image.composite(im, Image.new('RGB', im.size, 'white'), im)
    pps = im.size[0]/n_squares
    for i in range(n_squares):
        row = list()
        for j in range(n_squares):
            boxOuter = (j*pps,i*pps,j*pps+pps,i*pps+pps)
            tile = im.crop(boxOuter)
            if not tile.getbbox():
                tileText = " "
            else:
                tileInner = tile.crop((7,9,27,27))
                tileText = pytesseract.image_to_string(tileInner,config="-psm 10 -l eng -c tessedit_char_whitelist="+string.ascii_uppercase)
                tileText = "I" if tileText == "" else tileText
            row.append(tileText)
        print(" ".join(row))
parseAndPrintImageDemo(15, './images/1002-17.png')

F I S H   F L I P   H A R S H
O H H I   E O N S   A L I K E
C O O P   M U S I C N O T E S
A P T   F U N     U S N E W S
L E T T E R G R A D E S      
    O E R   E E L   L O F T S
S C H E M E   C A N     R I A
C H E M I C A L S Y M B O L S
A I L     O U I   C E A S E S
B A L E S   D N A   A L T    
      M O V I E R A T I N G S
M A R B L E     T I S   I R A
B L O O D T Y P E S   A X E L
A B O D E   E A R L   H O E S
S A T Y R   S T Y E   A N K A


### How far it gets with one puzzle
* Less than half of clues have the correct answer among their candidates
* Monday, October 2, 2017: 42%
* Difficult to find the correct answer by cross-referencing

In [24]:
cand_methods = ['cand_kg','cand_wk','cand_vec','cand_wn','cand_dct']

def getHitCountDemo(clues):
    success = []
    tally = []
    for i in clues:
        score = [0 for x in cand_methods]
        add = False
        for index,j in enumerate(cand_methods):
            if i.get(j) and re.sub(" ","",i['answer'].lower()) in [x.lower() for x in i[j]]:
                add = True
                score[index] = 1
        if add:
            success.append(i)
            tally.append(score)
    print('hits:',len(success),'/',len(clues),'=',len(success)/len(clues))
    return success,tally

puzzle = json.load(open('./data/merge_1002-17_cands.json','r'))
clues = puzzle['clues']
success,tally = getHitCountDemo(clues)

hits: 32 / 76 = 0.42105263157894735


### Which methods are generating hits?

In [40]:
df = pd.DataFrame(tally,index=map(lambda x: x['answer'],success),columns=cand_methods)
df.sum(0)

cand_kg      7
cand_wk     24
cand_vec     1
cand_wn      5
cand_dct     3
dtype: int64

In [41]:
df

,cand_kg,cand_wk,cand_vec,cand_wn,cand_dct
FISH,0,1,0,0,0
FLIP,0,0,0,1,0
APT,0,0,0,0,1
FUN,0,0,0,1,1
SCHEME,0,0,1,0,0
CAN,0,1,0,0,0
CEASES,0,0,0,1,0
BALES,0,1,0,0,0
DNA,0,1,0,0,0
MARBLE,1,1,0,0,0


## V. Conclusion

### Cross-referencing
* Limited if less than 50% of answers are in the candidates
* Is there a threshold above which I can start winnowing down?

### Improving the hit rate
* More formulations (previously added ~10%)
* Other APIs? ML?
* Pattern to the non-hits
* Systems to handle multi-word answers, pairs of answers

## Thank You!

#### Source code: [github.com/anbnyc/xword](github.com/anbnyc/xword)

#### Find me: [@alecbarrett](www.twitter.com/alecbarrett)